In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import re
from pandas.api.types import is_numeric_dtype


In [11]:
#Cargamos los datasets
books = pd.read_csv('Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
users = pd.read_csv('Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

print("Books Data:    ", books.shape)
print("Users Data:    ", users.shape)
print("Books-ratings: ", ratings.shape)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields

Books Data:     (271360, 8)
Users Data:     (278858, 3)
Books-ratings:  (1149780, 3)


In [12]:
#Analizamos la bd de Books
print("Columns: ", list(books.columns))
books.head()

Columns:  ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']


,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg


In [13]:
#No necesitamos las columnas de imagenes
books.drop(['imageUrlS', 'imageUrlM', 'imageUrlL'], axis=1, inplace=True)


In [14]:
books.isnull().sum()

ISBN                 0
bookTitle            0
bookAuthor           1
yearOfPublication    0
publisher            2
dtype: int64

In [15]:
#Corregimos
books.at[187689 ,'bookAuthor'] = 'Other'

books.at[128890 ,'publisher'] = 'Other'
books.at[129037 ,'publisher'] = 'Other'

pd.set_option('display.max_colwidth', None)
books.at[209538 ,'publisher'] = 'DK Publishing Inc'
books.at[209538 ,'yearOfPublication'] = 2000
books.at[209538 ,'bookTitle'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books.at[209538 ,'bookAuthor'] = 'Michael Teitelbaum'

books.at[221678 ,'publisher'] = 'DK Publishing Inc'
books.at[221678 ,'yearOfPublication'] = 2000
books.at[209538 ,'bookTitle'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books.at[209538 ,'bookAuthor'] = 'James Buckley'

books.at[220731 ,'publisher'] = 'Gallimard'
books.at[220731 ,'yearOfPublication'] = '2003'
books.at[209538 ,'bookTitle'] = 'Peuple du ciel - Suivi de Les bergers '
books.at[209538 ,'bookAuthor'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

books['yearOfPublication'] = books['yearOfPublication'].astype(int)
books.loc[books['yearOfPublication'] > 2021, 'yearOfPublication'] = 2002
books.loc[books['yearOfPublication'] == 0, 'yearOfPublication'] = 2002

In [16]:
books['ISBN'] = books['ISBN'].str.upper()
books.drop_duplicates(keep='last', inplace=True) 
books.reset_index(drop = True, inplace = True)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271047 entries, 0 to 271046
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ISBN               271047 non-null  object
 1   bookTitle          271047 non-null  object
 2   bookAuthor         271047 non-null  object
 3   yearOfPublication  271047 non-null  int64 
 4   publisher          271047 non-null  object
dtypes: int64(1), object(4)
memory usage: 10.3+ MB


In [17]:
required = users[users['Age'] <= 90]
required = required[required['Age'] >= 10]

mean = round(required['Age'].mean())

users.loc[users['Age'] > 80, 'Age'] = mean
users.loc[users['Age'] < 10, 'Age'] = mean
users['Age'] = users['Age'].fillna(mean)
users['Age'] = users['Age'].astype(int)

In [18]:
#Para corregir la columna de ubicación, dividimos en dif columnas
list_ = users.Location.str.split(', ')

city = []
state = []
country = []
count_no_state = 0    
count_no_country = 0

#Quitamos entradas inválidas y las reemplazamos por Other
for i in range(0,len(list_)):
    if list_[i][0] == ' ' or list_[i][0] == '' or list_[i][0]=='n/a' or list_[i][0] == ',': 
        city.append('other')
    else:
        city.append(list_[i][0].lower())

    if(len(list_[i])<2):
        state.append('other')
        country.append('other')
        count_no_state += 1
        count_no_country += 1
    else:
        if list_[i][1] == ' ' or list_[i][1] == '' or list_[i][1]=='n/a' or list_[i][1] == ',':  
            state.append('other')
            count_no_state += 1            
        else:
            state.append(list_[i][1].lower())
        
        if(len(list_[i])<3):
            country.append('other')
            count_no_country += 1
        else:
            if list_[i][2] == ''or list_[i][1] == ',' or list_[i][2] == ' ' or list_[i][2] == 'n/a':
                country.append('other')
                count_no_country += 1
            else:
                country.append(list_[i][2].lower())
        
users = users.drop('Location',axis=1)

#Corregimos cuando lugares ya fueron especificados en otra columna
temp = []
for ent in city:
    c = ent.split('/') 
    temp.append(c[0])

df_city = pd.DataFrame(temp,columns=['City'])
df_state = pd.DataFrame(state,columns=['State'])
df_country = pd.DataFrame(country,columns=['Country'])

users = pd.concat([users, df_city], axis=1)
users = pd.concat([users, df_state], axis=1)
users = pd.concat([users, df_country], axis=1)

In [19]:
users.drop_duplicates(keep='last', inplace=True)
users.reset_index(drop=True, inplace=True)

In [20]:
#Revisamos sintaxis de ISBN
flag = 0
k =[]
reg = "[^A-Za-z0-9]"

for x in ratings['ISBN']:
    z = re.search(reg,x)    
    if z:
        flag = 1

if flag == 1:
    print("False")
else:
    print("True")

False


In [21]:
#Los valores extra de ISBN los quitamos cuando ya existan en el ds de Books
bookISBN = books['ISBN'].tolist() 
reg = "[^A-Za-z0-9]" 
for index, row_Value in ratings.iterrows():
    z = re.search(reg, row_Value['ISBN'])    
    if z:
        f = re.sub(reg,"",row_Value['ISBN'])
        if f in bookISBN:
            ratings.at[index , 'ISBN'] = f

In [22]:
ratings['ISBN'] = ratings['ISBN'].str.upper()

ratings.drop_duplicates(keep='last', inplace=True)
ratings.reset_index(drop=True, inplace=True)

In [23]:
ratings.head()

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [24]:
ratings_valid = ratings[ratings['bookRating'] != 0]
ratings_valid = ratings_valid.reset_index(drop = True)
ratings_valid.shape
ratings_valid.head()

,userID,ISBN,bookRating
0,276726,0155061224,5
1,276729,052165615X,3
2,276729,0521795028,6
3,276736,3257224281,8
4,276737,0600570967,6


In [25]:
rating_count = pd.DataFrame(ratings_valid.groupby('ISBN')['bookRating'].count())
rating_count.sort_values('bookRating', ascending=False).head()

,bookRating
ISBN,
0316666343,707
0971880107,581
0385504209,488
0312195516,383
0679781587,333


In [26]:
most_rated_books = pd.DataFrame(['0316666343', '0971880107', '0385504209', '0312195516', '0679781587'], index=np.arange(5), columns = ['ISBN'])
most_rated_books_summary = pd.merge(most_rated_books, books, on='ISBN')
most_rated_books_summary

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
1,0971880107,Wild Animus,Rich Shapero,2004,Too Far
2,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday
3,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA


## Recomendación con Correlación

In [27]:
average_rating = pd.DataFrame(ratings_valid.groupby('ISBN')['bookRating'].mean())
average_rating['ratingCount'] = pd.DataFrame(ratings_valid.groupby('ISBN')['bookRating'].count())
average_rating.sort_values('ratingCount', ascending=False).head()

,bookRating,ratingCount
ISBN,,
0316666343,8.185290,707
0971880107,4.390706,581
0385504209,8.426230,488
0312195516,8.182768,383
0679781587,8.408408,333


In [28]:
#Usar usuarios con 200 o más ratings y libros con 100 o más ratings
counts1 = ratings_valid['userID'].value_counts()
ratings_valid = ratings_valid[ratings_valid['userID'].isin(counts1[counts1 >= 200].index)]
counts = ratings_valid['bookRating'].value_counts()
ratings_valid = ratings_valid[ratings_valid['bookRating'].isin(counts[counts >= 100].index)]

In [29]:
#Crear Matrix 
ratings_pivot = ratings_valid.pivot(index='userID', columns='ISBN').bookRating
userID = ratings_pivot.index
ISBN = ratings_pivot.columns
print(ratings_pivot.shape)
ratings_pivot.head()

(145, 53994)


ISBN,9022906116,0.330241664,0000000000,00000000000,0000000051,0000018030,0000913154,0001046438,000104687X,0001047213,...,O425155404,O439060737,O446611638,O590418262,O67174142X,O9088446X,X000000000,ZR903CX0003,"\0432534220\""""","\2842053052\"""""
userID,,,,,,,,,,,,,,,,,,,,,
2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#Test con el libro con más ratings (Lovely Bones)
booktest_ratings = ratings_pivot['0316666343']
similar_to_book = ratings_pivot.corrwith(booktest_ratings)
corr_book = pd.DataFrame(similar_to_book, columns=['pearsonR'])
corr_book.dropna(inplace=True)
corr_summary = corr_book.join(average_rating['ratingCount'])
corr_summary[corr_summary['ratingCount']>=300].sort_values('pearsonR', ascending=False).head(10)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,pearsonR,ratingCount
ISBN,,
0316666343,1.000000,707
0060928336,1.000000,320
0142001740,0.866154,314
0679781587,0.397360,333
0385504209,-0.117946,488
0312195516,-0.513239,383
059035342X,-0.944911,315
0971880107,-0.981981,581


In [32]:
books_corr_to_booktest = pd.DataFrame(['0316666343', '0060928336', '0142001740', '0679781587', '0385504209', '0312195516', '059035342X', '0971880107'], 
                                  index=np.arange(8), columns=['ISBN'])
corr_books = pd.merge(books_corr_to_booktest, books, on='ISBN')
corr_books

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
1,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
2,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
3,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday
4,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
5,059035342X,Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)),J. K. Rowling,1999,Arthur A. Levine Books
6,0971880107,Wild Animus,Rich Shapero,2004,Too Far


## Recomendación kNN

In [35]:
combine_book_rating = pd.merge(ratings_valid, books, on='ISBN')
columns = ['yearOfPublication', 'publisher', 'bookAuthor']
combine_book_rating = combine_book_rating.drop(columns, axis=1)
combine_book_rating.head()

,userID,ISBN,bookRating,bookTitle
0,2276,0020960808,10,The Natural Superiority of Women
1,2276,0030632366,9,Outrageous Acts and Everyday Rebellions
2,2276,0061030643,8,The Associate
3,11676,0061030643,7,The Associate
4,172742,0061030643,7,The Associate


In [36]:
#Agrupamos por título y creamos un Rating Count Total
combine_book_rating = combine_book_rating.dropna(axis = 0, subset = ['bookTitle'])

book_ratingCount = (combine_book_rating.
     groupby(by = ['bookTitle'])['bookRating'].
     count().
     reset_index().
     rename(columns = {'bookRating': 'totalRatingCount'})
     [['bookTitle', 'totalRatingCount']]
    )
book_ratingCount.head()

,bookTitle,totalRatingCount
0,Dark Justice,1
1,Final Fantasy Anthology: Official Strategy Guide (Brady Games),1
2,"Good Wives: Image and Reality in the Lives of Women in Northern New England, 1650-1750",1
3,Highland Desire (Zebra Splendor Historical Romances),1
4,Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),2


In [37]:
#Combinamos con el total Rating count para saber los libros más populares
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on = 'bookTitle', right_on = 'bookTitle', how = 'left')
rating_with_totalRatingCount.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount
0,2276,0020960808,10,The Natural Superiority of Women,1
1,2276,0030632366,9,Outrageous Acts and Everyday Rebellions,3
2,2276,0061030643,8,The Associate,4
3,11676,0061030643,7,The Associate,4
4,172742,0061030643,7,The Associate,4


In [38]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   44309.000
mean        1.405
std         1.243
min         1.000
25%         1.000
50%         1.000
75%         1.000
max        30.000
Name: totalRatingCount, dtype: float64


In [39]:
#Como el libro en la mediana solo se calificó una vez, chequemos la distibución superior
print(book_ratingCount['totalRatingCount'].quantile(np.arange(.9, 1, .01)))

0.900   2.000
0.910   2.000
0.920   2.000
0.930   3.000
0.940   3.000
0.950   3.000
0.960   4.000
0.970   4.000
0.980   5.000
0.990   7.000
Name: totalRatingCount, dtype: float64


In [40]:
popularity_threshold = 7
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount
50,2276,0380002930,9,Watership Down,18
51,6575,0380002930,4,Watership Down,18
52,11676,0380002930,5,Watership Down,18
53,16634,0380002930,10,Watership Down,18
54,16795,0380002930,6,Watership Down,18


In [41]:
combined = rating_popular_book.merge(users, left_on = 'userID', right_on = 'userID', how = 'left')
user_rating=combined.drop('Age', axis=1)
user_rating.head()

,userID,ISBN,bookRating,bookTitle,totalRatingCount,City,State,Country
0,2276,0380002930,9,Watership Down,18,niskayuna,new york,usa
1,6575,0380002930,4,Watership Down,18,seattle,washington,usa
2,11676,0380002930,5,Watership Down,18,other,other,other
3,16634,0380002930,10,Watership Down,18,little rock,arkansas,usa
4,16795,0380002930,6,Watership Down,18,mechanicsville,maryland,usa


In [42]:
#Convertimos a Matrix y llenamos con 0 los null para calcular distancias

user_rating = user_rating.drop_duplicates(['userID', 'bookTitle'])
user_rating_pivot = user_rating.pivot(index = 'bookTitle', columns = 'userID', values = 'bookRating').fillna(0)
user_rating_matrix = csr_matrix(user_rating_pivot.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(user_rating_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [64]:
query_index = np.random.choice(user_rating_pivot.shape[0])
distances, indices = model_knn.kneighbors(user_rating_pivot.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, user_rating_pivot.index[indices.flatten()[i]], distances.flatten()[i]))
        

Recommendations for Final Target:

1: Body of Lies, with distance of 0.3941190981965006:
2: The Beach House, with distance of 0.41400413623782617:
3: By the Light of the Moon, with distance of 0.4184274507786191:
4: Shock, with distance of 0.42499281236523156:
5: The Mulberry Tree, with distance of 0.4388361412499666:


## Matrix Factorization con SVD 

In [46]:
user_rating_pivot2 = user_rating.pivot(index = 'userID', columns = 'bookTitle', values = 'bookRating').fillna(0)
user_rating_pivot2.head()

bookTitle,16 Lighthouse Road,1984,1st to Die: A Novel,2nd Chance,84 Charing Cross Road,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Confederacy of Dunces (Evergreen Book),A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Knight in Shining Armor,...,Wish You Well,"Wizard and Glass (The Dark Tower, Book 4)","Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",Wuthering Heights,Xanth 15: The Color of Her Panties,Year of Wonders,You Belong to Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
userID,,,,,,,,,,,,,,,,,,,,,
2276,0.000,0.000,0.000,10.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3757,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4385,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6242,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,7.000,0.000,0.000,0.000,0.000
6251,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [47]:
#Trasponemos esta matriz para que bookTitles se hagan Rows 
#y los Users sean las columnas
user_rating_pivot2.shape

(139, 502)

In [48]:
X = user_rating_pivot2.values.T
X.shape

(502, 139)

In [49]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(502, 12)

In [50]:
#Calculamos el Coeficiente de Correlacion de Pearson R para cada libro
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(502, 502)

In [65]:
#Probamos con cualquier libro
book_title = user_rating_pivot2.columns
book_list = list(book_title)
book_test = input("Libro a buscar : ")
recomm = book_list.index(book_test)
print(recomm)

Libro a buscar : Final Target
130


In [66]:
corr_recommend = corr[recomm]

In [67]:
corr_recommend.shape

(502,)

In [68]:
print("Recomendaciones: ")
list(book_title[(corr_recommend<1.0) & (corr_recommend > 0.8)][1:])

Recomendaciones: 


['Between Friends',
 'Body of Lies',
 'Chosen Prey',
 "Daddy's Little Girl",
 'Envy',
 'Fast Women',
 'Fever',
 'Flashback',
 'Four Blind Mice',
 'Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel)',
 'In Her Shoes : A Novel',
 'Mortal Prey',
 'Night Sins',
 'Open Season',
 'Over the Edge',
 'Q Is for Quarry',
 'Shock',
 'The Beach House',
 'The Oath',
 'The Switch',
 'To the Nines: A Stephanie Plum Novel',
 'Unnatural Exposure']